SETUP AND INSTALLATION

In [1]:
!nvidia-smi

Sat Aug 29 20:57:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash
cd /content/
git clone https://github.com/dcyphr/dcyphr-NLP.git


Cloning into 'dcyphr-NLP'...


In [3]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install -q -e .
%cd examples
!pip install -U -q -r requirements.txt
!pip install -U pyarrow

Cloning into 'transformers'...
remote: Enumerating objects: 39728, done.
remote: Total 39728 (delta 0), reused 0 (delta 0), pack-reused 39728
Receiving objects: 100% (39728/39728), 28.55 MiB | 25.85 MiB/s, done.
Resolving deltas: 100% (27585/27585), done.
/content/transformers
     |████████████████████████████████| 3.0MB 4.4MB/s 
     |████████████████████████████████| 1.1MB 13.2MB/s 
     |████████████████████████████████| 890kB 15.6MB/s 
/content/transformers/examples
     |████████████████████████████████| 6.8MB 4.9MB/s 
     |████████████████████████████████| 460kB 48.9MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 3.4MB 32.1MB/s 
     |████████████████████████████████| 317kB 55.6MB/s 
     |████████████████████████████████| 11.6MB 52.5MB/s 
     |████████████████████████████████| 4.7MB 57.8MB/s 
     |████████████████████████████████| 7.2MB 40.8MB/s 
     |████████████████████████████████| 225kB 57.7MB/s 
     |████████████████

In [7]:
%cd /content/transformers/examples/seq2seq/

/content/transformers/examples/seq2seq


This installs the XSUM Dataset and is unneccesary if you are working with your own dataset

In [ ]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/summarization/xsum.tar.gz
!tar -xzvf xsum.tar.gz
!export XSUM_DIR=${PWD}/xsum

--2020-08-29 17:14:32--  https://s3.amazonaws.com/datasets.huggingface.co/summarization/xsum.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.1.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.1.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204844092 (195M) [application/x-tar]
Saving to: ‘xsum.tar.gz’

xsum.tar.gz         100%[===================>] 195.35M  44.8MB/s    in 4.8s    

2020-08-29 17:14:37 (40.5 MB/s) - ‘xsum.tar.gz’ saved [204844092/204844092]

xsum/
xsum/train.target
xsum/train.source
xsum/val.source
xsum/val.target
xsum/test.source
xsum/test.target


This Finetunes the PEGASUS Model for One epoch on the Dcyphr fine-tuning data

In [8]:
!bash finetune_pegasus_xsum.sh \
  --model_name_or_path google/pegasus-xsum \
  --data_dir /content/dcyphr-NLP/fine_tuning_data \
  --output_dir xsum_pegasus_test_5 \
  --train_batch_size 2 \
  --eval_batch_size 2 \
  --num_train_epochs 1 \
  --n_train 100 \
  --n_val 100 \
  --sortish_sampler \
  --gpus 1 \
  --val_check_interval 0.25 \
  --gradient_accumulation_steps 4 \

PyTorch version 1.6.0+cu101 available.
2020-08-29 20:59:44.133297: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorFlow version 2.3.0 available.
https://s3.amazonaws.com/models.huggingface.co/bert/google/pegasus-xsum/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpuc9e5ea6
storing https://s3.amazonaws.com/models.huggingface.co/bert/google/pegasus-xsum/config.json in cache at /root/.cache/torch/transformers/b5be2b326342de3aafd13a93209b8d485791b202dcc54707b1b9c0971baa0a0e.f45c72b25fcb58b55589c0dfccde9d4314362d7297bb2bfd980655f4a3cbe3b7
creating metadata file for /root/.cache/torch/transformers/b5be2b326342de3aafd13a93209b8d485791b202dcc54707b1b9c0971baa0a0e.f45c72b25fcb58b55589c0dfccde9d4314362d7297bb2bfd980655f4a3cbe3b7
loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/google/pegasus-xsum/config.json from cache at /root/

In [9]:
%%bash
mkdir gens
export DATA_DIR=xsum
python run_eval.py google/pegasus-xsum \
    $DATA_DIR/test.source gens/peg_xsum_test_generation.txt \
    --reference_path $DATA_DIR/test.target \
    --score_path gens/peg_xsum_rouge.txt --task summarization \
    --device cuda \
    --bs 8

PyTorch version 1.6.0+cu101 available.
2020-08-29 21:08:23.895870: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorFlow version 2.3.0 available.
Traceback (most recent call last):
  File "run_eval.py", line 137, in <module>
    run_generate()
  File "run_eval.py", line 104, in run_generate
    examples = [" " + x.rstrip() if "t5" in args.model_name else x.rstrip() for x in open(args.input_path).readlines()]
FileNotFoundError: [Errno 2] No such file or directory: 'xsum/test.source'


**Training on Dcyphr Data**

Command to Finetune on the dcyphr dataset

In [10]:
%%bash
export DATA_DIR=/content/dcyphr-NLP/fine_tuning_data
python run_eval.py sshleifer/distilbart-cnn-12-6 $DATA_DIR/val.source dbart_val_generations.txt \
    --reference_path $DATA_DIR/val.target \
    --score_path cnn_rouge.json \
    --task summarization \
    --n_obs 100 \
    --device cuda \
    --bs 32

{'rouge1': 32.9453, 'rouge2': 9.8099, 'rougeL': 19.7652, 'n_obs': 20, 'runtime': 13.766160488128662, 'seconds_per_sample': 0.6883}


PyTorch version 1.6.0+cu101 available.
2020-08-29 21:08:32.962542: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorFlow version 2.3.0 available.
https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/distilbart-cnn-12-6/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp2ce3wln4
storing https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/distilbart-cnn-12-6/config.json in cache at /root/.cache/torch/transformers/90dbb6d4e8ba2227a79a0e5cc64eb216091a1d5c362a83c52c6c242a44d89b50.e1484cfc83ca98dbf8dbb660097e95a5ef503f348dd354064e58fa28f5e3ae16
creating metadata file for /root/.cache/torch/transformers/90dbb6d4e8ba2227a79a0e5cc64eb216091a1d5c362a83c52c6c242a44d89b50.e1484cfc83ca98dbf8dbb660097e95a5ef503f348dd354064e58fa28f5e3ae16
loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/distilbart-cnn-12-6/co

In [11]:
%%bash
ls

bertabs
callbacks.py
cnn_rouge.json
convert_model_to_fp16.py
dbart_val_generations.txt
distillation.py
download_wmt.py
finetune_bart_tiny.sh
finetune_pegasus_xsum.sh
finetune.py
finetune.sh
finetune_t5.sh
gens
initialization_utils.py
__init__.py
lightning_logs
minify_dataset.py
pack_dataset.py
__pycache__
README.md
romanian_postprocessing.md
run_distiller.sh
run_eval.py
test_bash_script.py
test_data
test_seq2seq_examples.py
train_distilbart_cnn.sh
train_distilbart_xsum.sh
train_mbart_cc25_enro.sh
utils.py
xsum_pegasus_test_5


Okay so the output of the data can be found at transformers/examples/seq2seq/dbart_val_generation